# Spotify Data Collection

We need data for the clustering analysis and the recommendation engine.

We will collect them seperately.

In [ ]:
import pandas as pd
import spotipy
import spotipy.util as util
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import json
import time
import random 
from tqdm import tqdm

In [ ]:
# Go online to the API site to collect relevant info to replace xxxxxx below
username = xxxxxxxx
scope = 'user-library-read'
client_id = xxxxxxxx
client_secret = xxxxxxxx

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
# Read in albums to scrape as a dataframe with album name and artist name here:
'''
.....
'''

In [ ]:
def find_sound_features_from_album(album_name, artist_name):
    """
    Note: Will only return a single album's audio features per request
    """

    q = ("artist:{} album:{}".format(album_name, artist_name))
    
    try:
        # Random sleep to keep API working well
        time.sleep((random.randint(1,10)/10))

        # Create query
        results = sp.search(q=q, type='album',limit=1)

        # Collect album URI
        uri = results['albums']['items'][0]['uri']

        # Fetch album tracks
        album = sp.album_tracks(uri)

        # Extract relevant song names and ids
        song_names = []
        song_ids = []
        for song in album['items']:
            song_names.append(song['name'])
            song_ids.append(song['id'])

        # Populate audio features per song
        audio_features = []
        for s in range(len(song_ids)):
            audio_features.append(sp.audio_features(song_ids[s])[0])

        # Add to dataframe
        temp_df = pd.DataFrame({'song_name': song_names,
                             'song_id': song_ids,
                             'features': audio_features

        })
        
        # Append album and artist to dataframe
        temp_df['album'] = album_name
        temp_df['artist'] = artist_name
        
        return pd.concat([temp_df.drop(['features'], axis=1), temp_df['features'].apply(pd.Series)], axis = 1)
         
    except:
        print("Couldn't find {}".format(q))
        

In [ ]:
# Create tuples of album name and artist name from data frame
album_artist_tuples = [tuple(x) for x in to_collect[['album_name', 'artist_name']].to_numpy()]

# Collect features
all_audio_features = []
for i, v in tqdm(enumerate(album_artist_tuples)):
    all_audio_features.append(find_sound_features_from_album(v[1], v[0]))

In [ ]:
# Concat all dataframes into one
extra_spotify_features = all_audio_features[0]
for i in range(len(all_audio_features)):
    extra_spotify_features = pd.concat([extra_spotify_features, all_audio_features[i]], axis = 0)

In [ ]:
# Check it worked
extra_spotify_features.head()

In [ ]:
# Export
extra_spotify_features.to_csv('[insert name here]', index=False)